In [ ]:
#!git clone https://gitlab.lrz.de/arl-ws2021/gym-crawlingrobot.git
#!pip install -e ../.

# Manual

In [11]:
import sys
import numpy as np
import pygame
import os
#os.environ['SDL_VIDEODRIVER']='dummy'
import gym
import gym_crawlingrobot

In [12]:
env = gym.make('crawlingrobot-discrete-v1', rotation_angles=5, goal_distance=1300, window_size=(1500, 800), render_intermediate_steps=True, plot_steps_per_episode=True)
env.robot.mode = 2
done = False
terminate = False
action = None
env.reset()

while not terminate:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            terminate = True
            continue
            #sys.exit(0)
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()
                terminate = True
                continue
                #sys.exit(0)
            elif event.key == pygame.K_UP or event.key == pygame.K_w:
                env.step(0)
                action = 0
            elif event.key == pygame.K_RIGHT or event.key == pygame.K_d:
                action = 1
            elif event.key == pygame.K_DOWN or event.key == pygame.K_s:
                action = 2
            elif event.key == pygame.K_LEFT or event.key == pygame.K_a:
                action = 3
            elif event.key == pygame.K_r:
                env.reset()
                action = 3
            elif event.key == pygame.K_SPACE:
                env.robot.render_intermediate_steps = not env.robot.render_intermediate_steps

            if action:
                obs, rewards, done, info = env.step(action)
                action = None
            if done:
                env.reset()
                action = 3

    if not terminate:
        env.render()
    else: 
        print ("exiting ... ")


exiting ... 


# Q-Learning

In [13]:
np.set_printoptions(threshold=sys.maxsize)

alpha = 0.3
gamma = 0.9
epsilon = 0.15
maxSteps = 10000
episodes = 200


env = gym.make('crawlingrobot-discrete-v1', rotation_angles=5, goal_distance=2500, window_size=(1500, 800), plot_steps_per_episode=True)
max = env.observation_space.high[0] + 1  # aktuell 5

Q = np.zeros([max ** len(env.observation_space.high), env.action_space.n])
terminate = False

for episode in range (episodes):
    done = False
    state = env.reset()
    step = 0

    print("Episode", episode, "started...")

    while not done and step < maxSteps and not terminate:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    terminate = True
                    #sys.exit(0)
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        pygame.quit()
                        terminate = True
                        #sys.exit(0)
                    if event.key == pygame.K_SPACE:
                        env.robot.render_intermediate_steps = not env.robot.render_intermediate_steps

            
            if not terminate:
                # action selection
                if np.random.rand() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = np.argmax(Q[state])

                # perform action in environment
                nextObs, reward, done, _ = env.step(action)
                env.render()
                nextState = int(nextObs[0] * max + nextObs[1])

                # Q-learning
                Q[state, action] += alpha * (reward + gamma * np.max(Q[nextState]) - Q[state, action])

                # time transition
                state = nextState
                step += 1
            else: 
                print ("exiting ... ")
                break


    if not terminate: 
        res = 0
        if len(env.robot.episode_time_results) > 0:
            res = env.robot.episode_time_results[-1]
        print("Episode", episode, "took", step, "steps, took", res,"simulation seconds")
    else:
        pygame.quit()
        break


Episode 0 started...
Episode 0 took 1213 steps, took 216.29 simulation seconds
Episode 1 started...
exiting ... 


# PPO2



In [ ]:
#!pip install stable-baselines
#!pip install tensorflow==1.15

In [14]:
import pygame
import sys
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.bench import Monitor
from stable_baselines import PPO2
import gym
import gym_crawlingrobot

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

c:\users\z003c4dc\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/m

In [ ]:
log_dir = ""
env = gym.make('crawlingrobot-continuous-v1', goal_distance=2500, window_size=(1500, 800), render_intermediate_steps=False)
robot_env = env
ppo2 = "ppo2"
os.makedirs(ppo2, exist_ok=True)

def learn():
    global env
    env = VecNormalize(DummyVecEnv([lambda: Monitor(env, log_dir)]), norm_obs=True, norm_reward=False)
    model = PPO2(MlpPolicy, env, verbose=1, learning_rate=0.15)
    model.learn(total_timesteps=30000, callback=callback)
    model.save("ppo2/ppo2_crawling_robot")
    env.save("ppo2/vec_normalize.pkl")
 
    del model, env

def callback(a, b):
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit(0)
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                pygame.quit()
                sys.exit(0)
            if event.key == pygame.K_SPACE:
                robot_env.robot.render_intermediate_steps = not robot_env.robot.render_intermediate_steps

    env.render()

def show():
    global env
    env = DummyVecEnv([lambda: Monitor(env, log_dir)])
    env = VecNormalize.load("ppo2/vec_normalize.pkl", env)
    model = PPO2.load("ppo2/ppo2_crawling_robot")

    env.training = False
    env.norm_reward = False

    obs = env.reset()
    done = False

    while True:
        if done:
            env.reset()

        action, _states = model.predict(obs)
        print(action)
        obs, rewards, done, info = env.step(action)
        callback(None, None)


learn()
#show()
pygame.quit()


c:\users\z003c4dc\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\gym\spaces\box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





----------------------------------
| approxkl           | 185237.34 |
| clipfrac           | 0.9277344 |
| explained_variance | -0.00106  |
| fps                | 84        |
| n_updates          | 1         |
| policy_entropy     | 2.2629325 |
| policy_loss        | 0.2847679 |
| serial_timesteps   | 128       |
| time_elapsed       | 0         |
| total_timesteps    | 128       |
| value_loss         | 2.1106427 |
----------------------------------
-----------------------------------
| approxkl           | 725445.4   |
| clipfrac           | 0.9375     |
| explained_variance | -11.2      |
| fps                | 121        |
| n_updates          | 2          |
| policy_entropy     | 3.4800706  |
| policy_loss        | 0.34924755 |
| serial_timesteps   | 256        |
| time_elapsed       | 1.51       |
| total_timesteps    | 256        |
| value_loss         | 1.0308319  |
-----------------------------------
-----------------------------------
| approxkl           | 897.6333   |
| cli

-------------------------------------
| approxkl           | 0.081736594  |
| clipfrac           | 0.29882812   |
| explained_variance | -0.035       |
| fps                | 121          |
| n_updates          | 18           |
| policy_entropy     | 8.022804     |
| policy_loss        | -0.010093376 |
| serial_timesteps   | 2304         |
| time_elapsed       | 24           |
| total_timesteps    | 2304         |
| value_loss         | 19.67577     |
-------------------------------------
------------------------------------
| approxkl           | 0.13903652  |
| clipfrac           | 0.41015625  |
| explained_variance | -0.0221     |
| fps                | 121         |
| n_updates          | 19          |
| policy_entropy     | 7.9764433   |
| policy_loss        | 0.034374125 |
| serial_timesteps   | 2432        |
| time_elapsed       | 25          |
| total_timesteps    | 2432        |
| value_loss         | 10.4221525  |
------------------------------------
-------------------------

------------------------------------
| approxkl           | 0.081138164 |
| clipfrac           | 0.6347656   |
| explained_variance | 0.0401      |
| fps                | 120         |
| n_updates          | 36          |
| policy_entropy     | 8.861767    |
| policy_loss        | 0.042585313 |
| serial_timesteps   | 4608        |
| time_elapsed       | 44.4        |
| total_timesteps    | 4608        |
| value_loss         | 13.7229805  |
------------------------------------
------------------------------------
| approxkl           | 0.013058301 |
| clipfrac           | 0.19335938  |
| explained_variance | 0.0463      |
| fps                | 115         |
| n_updates          | 37          |
| policy_entropy     | 9.026958    |
| policy_loss        | 0.009081487 |
| serial_timesteps   | 4736        |
| time_elapsed       | 45.5        |
| total_timesteps    | 4736        |
| value_loss         | 43.49103    |
------------------------------------
------------------------------------
|

--------------------------------------
| approxkl           | 0.0022518618  |
| clipfrac           | 0.02734375    |
| explained_variance | 0.0932        |
| fps                | 111           |
| n_updates          | 53            |
| policy_entropy     | 10.497949     |
| policy_loss        | -0.0009041199 |
| serial_timesteps   | 6784          |
| time_elapsed       | 63.3          |
| total_timesteps    | 6784          |
| value_loss         | 252.00586     |
--------------------------------------
--------------------------------------
| approxkl           | 0.0007956796  |
| clipfrac           | 0.005859375   |
| explained_variance | 0.0119        |
| fps                | 113           |
| n_updates          | 54            |
| policy_entropy     | 10.510279     |
| policy_loss        | -0.0043324293 |
| serial_timesteps   | 6912          |
| time_elapsed       | 64.5          |
| total_timesteps    | 6912          |
| value_loss         | 1694.0118     |
-------------------------